In [ ]:
!pip install Wordcloud
!pip install Sastrawi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.naive_bayes import MultinomialNB
from wordcloud import WordCloud, STOPWORDS

In [ ]:
data = pd.read_csv('Ganjar.csv')
data.head()

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
0,Sun Aug 27 22:47:27 +0000 2023,1.695930e+18,Gubernur Jawa Tengah Ganjar Pranowo berkomitme...,0,0,20,24,in,1.226730e+18,1.695930e+18,aan_muba,https://twitter.com/aan_muba/status/1695931058...
1,Sun Aug 27 22:39:55 +0000 2023,1.695930e+18,GANJAR PEDULI DISABILITAS Gubernur Jawa Tenga...,0,1,19,26,in,1.226730e+18,1.695930e+18,aan_muba,https://twitter.com/aan_muba/status/1695929162...
2,Sun Aug 27 18:30:51 +0000 2023,1.695870e+18,"Dalam tahap akhir masa jabatannya, Gubernur Ja...",0,0,0,0,in,2.883691e+09,1.695870e+18,DPPBMI,https://twitter.com/DPPBMI/status/169586648317...
3,Sun Aug 27 17:02:58 +0000 2023,1.695840e+18,Dari desa Untuk Indoensia!! Ganjar Pranowo su...,0,0,7,9,in,2.996488e+09,1.695840e+18,catatankaqihaty,https://twitter.com/catatankaqihaty/status/169...
4,Sun Aug 27 16:26:54 +0000 2023,1.695840e+18,@BambangNoeryan4 @ekowboy2 @ganjarpranowo Ngur...,0,2,0,3,in,1.669740e+18,1.695810e+18,FurqanIbr64677,https://twitter.com/FurqanIbr64677/status/1695...


In [ ]:
data.dtypes

created_at              object
id_str                 float64
full_text               object
quote_count              int64
reply_count              int64
retweet_count            int64
favorite_count           int64
lang                    object
user_id_str            float64
conversation_id_str    float64
username                object
tweet_url               object
dtype: object

In [ ]:
data = data.astype(str)

In [ ]:
data.dtypes

created_at             object
id_str                 object
full_text              object
quote_count            object
reply_count            object
retweet_count          object
favorite_count         object
lang                   object
user_id_str            object
conversation_id_str    object
username               object
tweet_url              object
dtype: object

In [ ]:
data.shape

(1514, 12)

In [ ]:
data.isnull().sum()

created_at             0
id_str                 0
full_text              0
quote_count            0
reply_count            0
retweet_count          0
favorite_count         0
lang                   0
user_id_str            0
conversation_id_str    0
username               0
tweet_url              0
dtype: int64

In [ ]:
data.duplicated().sum()

0

In [ ]:
data.describe()

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
count,1514,1514,1514,1514,1514,1514,1514,1514,1514,1514,1514,1514
unique,1498,368,1479,9,27,69,106,1,330,370,334,1514
top,Sat Aug 19 03:00:01 +0000 2023,1.69274e+18,Jakarta â€” Elektabilitas Calon Presiden Mente...,0,0,0,0,in,1.23099e+18,1.69165e+18,sahabatganjar,https://twitter.com/aan_muba/status/1695931058...
freq,5,24,8,1418,866,846,641,1514,95,28,95,1


In [ ]:
data.head()

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
0,Sun Aug 27 22:47:27 +0000 2023,1.69593e+18,Gubernur Jawa Tengah Ganjar Pranowo berkomitme...,0,0,20,24,in,1.22673e+18,1.69593e+18,aan_muba,https://twitter.com/aan_muba/status/1695931058...
1,Sun Aug 27 22:39:55 +0000 2023,1.69593e+18,GANJAR PEDULI DISABILITAS Gubernur Jawa Tenga...,0,1,19,26,in,1.22673e+18,1.69593e+18,aan_muba,https://twitter.com/aan_muba/status/1695929162...
2,Sun Aug 27 18:30:51 +0000 2023,1.69587e+18,"Dalam tahap akhir masa jabatannya, Gubernur Ja...",0,0,0,0,in,2883691224.0,1.69587e+18,DPPBMI,https://twitter.com/DPPBMI/status/169586648317...
3,Sun Aug 27 17:02:58 +0000 2023,1.69584e+18,Dari desa Untuk Indoensia!! Ganjar Pranowo su...,0,0,7,9,in,2996488424.0,1.69584e+18,catatankaqihaty,https://twitter.com/catatankaqihaty/status/169...
4,Sun Aug 27 16:26:54 +0000 2023,1.69584e+18,@BambangNoeryan4 @ekowboy2 @ganjarpranowo Ngur...,0,2,0,3,in,1.66974e+18,1.69581e+18,FurqanIbr64677,https://twitter.com/FurqanIbr64677/status/1695...


In [ ]:
# Seleksi data Menghapus beberapa kolom dari DataFrame
columns_to_drop = ['created_at', 'id_str','quote_count','reply_count','retweet_count', 'favorite_count','lang', 'user_id_str','conversation_id_str']
data = data.drop(columns=columns_to_drop)
data.head()


,full_text,username,tweet_url
0,Gubernur Jawa Tengah Ganjar Pranowo berkomitme...,aan_muba,https://twitter.com/aan_muba/status/1695931058...
1,GANJAR PEDULI DISABILITAS Gubernur Jawa Tenga...,aan_muba,https://twitter.com/aan_muba/status/1695929162...
2,"Dalam tahap akhir masa jabatannya, Gubernur Ja...",DPPBMI,https://twitter.com/DPPBMI/status/169586648317...
3,Dari desa Untuk Indoensia!! Ganjar Pranowo su...,catatankaqihaty,https://twitter.com/catatankaqihaty/status/169...
4,@BambangNoeryan4 @ekowboy2 @ganjarpranowo Ngur...,FurqanIbr64677,https://twitter.com/FurqanIbr64677/status/1695...


In [ ]:
# Case Folding Mengubah teks menjadi lover data Text
data['full_text']= data['full_text'].str.lower()
data.head()

,full_text,username,tweet_url
0,gubernur jawa tengah ganjar pranowo berkomitme...,aan_muba,https://twitter.com/aan_muba/status/1695931058...
1,ganjar peduli disabilitas gubernur jawa tenga...,aan_muba,https://twitter.com/aan_muba/status/1695929162...
2,"dalam tahap akhir masa jabatannya, gubernur ja...",DPPBMI,https://twitter.com/DPPBMI/status/169586648317...
3,dari desa untuk indoensia!! ganjar pranowo su...,catatankaqihaty,https://twitter.com/catatankaqihaty/status/169...
4,@bambangnoeryan4 @ekowboy2 @ganjarpranowo ngur...,FurqanIbr64677,https://twitter.com/FurqanIbr64677/status/1695...


In [ ]:
#normalisasi
norm = {"aja":"saja","gak" : "tidak","yg" :"yang","sbg" : "sebagai","tau" : "tahu","becus" : "mampu","biayai" : "membiayai","digandrungi" : "disukai"}

def normalisasi(str_text):
  for i in norm:
    str_text = str_text.replace(i, norm[i])
  return str_text

data['full_text'] = data['full_text'].apply(lambda x: normalisasi (x))

In [ ]:
#Stopword
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
more_stop_word = []

stop_words = StopWordRemoverFactory().get_stop_words()
new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover (new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text
data['full_text'] = data['full_text'].apply(lambda x: stopword(x))
data.head()

,full_text,username,tweet_url
0,gubernur jawa tengah ganjar pranowo berkomitme...,aan_muba,https://twitter.com/aan_muba/status/1695931058...
1,ganjar peduli disabilitas gubernur jawa tenga...,aan_muba,https://twitter.com/aan_muba/status/1695929162...
2,"tahap akhir masa jabatannya, gubernur jawa ten...",DPPBMI,https://twitter.com/DPPBMI/status/169586648317...
3,desa indoensia!! ganjar pranowo sukses memban...,catatankaqihaty,https://twitter.com/catatankaqihaty/status/169...
4,@bambangnoeryan4 @ekowboy2 @ganjarpranowo ngur...,FurqanIbr64677,https://twitter.com/FurqanIbr64677/status/1695...


In [ ]:
#Tokenisasi
tokenized = data['full_text'].apply(lambda x:x.split())
tokenized

0       [gubernur, jawa, tengah, ganjar, pranowo, berk...
1       [ganjar, peduli, disabilitas, gubernur, jawa, ...
2       [tahap, akhir, masa, jabatannya,, gubernur, ja...
3       [desa, indoensia!!, ganjar, pranowo, sukses, m...
4       [@bambangnoeryan4, @ekowboy2, @ganjarpranowo, ...
                              ...                        
1509    [gubernur, jawa, tengah,, @ganjarpranowo, bers...
1510    [wali, kota, surakarta, gibran, rakabuming, ra...
1511    [wali, kota, surakarta, gibran, rakabuming, ra...
1512    [gubernur, jawa, tengah, ganjar, pranowo, meny...
1513    [gubernur, jawa, tengah, ganjar, pranowo, meng...
Name: full_text, Length: 1514, dtype: object

In [ ]:
# Steaming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(full_text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []  # Definisikan do di sini sebelum loop
    for w in full_text:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = " ".join(do)
    print (d_clean)
    return d_clean

# Menerapkan fungsi stemming pada kolom 'tokenized'
tokenized ['full_text'] = tokenized.apply(stemming)
tokenized.to_csv ('hasil_steaming.csv', index=False)
data_clean= pd.read_csv('hasil_steaming.csv', encoding ='latin1')

gubernur jawa tengah ganjar pranowo komitmen prioritas daya ekonomi perempuan kelompok rentan akar rumput cara lewat agam latih berwirausaha lebih daya perempuan kelompok rentan bukan laku umkm murni potret masyarakat https t co j03fnjhyangj
ganjar peduli disabilitas gubernur jawa tengah ganjar pranowo terus komitmen penuh hak sandang disabilitas cipta hidup nyaman aman ganjar kata hak diri lindung asuh sipil sehat sampai https t co xovdxfqj5j
tahap akhir masa jabat gubernur jawa tengah ganjar pranowo henti upaya tingkat mutu layan baik seluruh masyarakat jawa tengah salah satu fokus pada layan sedia air bersih jadi butuh vital https t co yjjd092dvm
desa indoensia ganjar pranowo sukses bangun jawa tengah ganjar bangun jateng rata mulai desa desa ini lah sebab jawa tengah jauh lebih maju 10 tahun ganjar jabat jadi gubernur jawa tengah https t co gw3tjaw8l7
bambangnoeryan4 ekowboy2 ganjarpranowo ngurus provinsi tidak mampu sesuai yang kata gagal ngurus provinsi jawa tengah youtuber kerja

In [ ]:
data_clean.head(100)

,full_text
0,"['gubernur', 'jawa', 'tengah', 'ganjar', 'pran..."
1,"['ganjar', 'peduli', 'disabilitas', 'gubernur'..."
2,"['tahap', 'akhir', 'masa', 'jabatannya,', 'gub..."
3,"['desa', 'indoensia!!', 'ganjar', 'pranowo', '..."
4,"['@bambangnoeryan4', '@ekowboy2', '@ganjarpran..."
...,...
95,"['kamis', '(24/8/2023),', 'gubernur', 'jawa', ..."
96,"['gubernur', 'jawa', 'tengah', 'ganjar', 'pran..."
97,"['@tita83079013', '@aniesbaswedan', '@ganjarpr..."
98,"['gubernur', 'jawa', 'tengahÃ¢', 'ganjar', 'pr..."
